<a href="https://colab.research.google.com/github/sulabh007/oral_cancer_detection_DAI/blob/main/ressnet50wab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import

In [ ]:
try:
    from torchinfo import summary
except:
    print("[INFO] Couldn't find torchinfo... installing it.")
    !pip install -q torchinfo
    from torchinfo import summary


[INFO] Couldn't find torchinfo... installing it.


In [ ]:
try:
    import wandb
except:
    !pip install wandb
    import wandb
wandb.login()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.4/196.4 kB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.7/257.7 kB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.2 MB/s eta 0:00:00


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
import pandas as pd
import seaborn as sns
import os
from pathlib import Path
import re
import random
import matplotlib.pyplot as plt
import math
import torch

from PIL import Image
from pandas import DataFrame
from typing import Tuple, Dict, List
from torch.utils.data import Dataset, DataLoader

"""
Contains functions for training and testing a PyTorch model.
"""
from torchinfo import summary
from tqdm.auto import tqdm
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay

from torchvision import datasets, models, transforms
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
drive = '/content/drive/MyDrive/'

Mounted at /content/drive


# Directory

In [ ]:
data_path = Path(drive)
image_path = data_path/'NDB-UFES An oral cancer and leukoplakia dataset composed of histopathological images and patient data'


In [ ]:
# Setup path for target directory
target_directory = image_path / 'patch'
print(f"Target directory: {target_directory}")

Target directory: /content/drive/MyDrive/NDB-UFES An oral cancer and leukoplakia dataset composed of histopathological images and patient data/patch


In [ ]:
image_dir = target_directory / 'images'
train_df = pd.read_csv(target_directory/'sabpatch_parsed_folders.csv')
train_ds = train_df[['path','lesion']]
test_df = pd.read_csv(target_directory/'sabpatch_parsed_test.csv')
test_ds = test_df[['path','lesion']]

# Global Parameters

In [ ]:
ALPHA = 0.0001 ## Learning Rate
EPOCH = 50  ## Epochs
BATCH_SIZE = 32
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

MEAN = [0.485, 0.456, 0.406]
STD = [0.229, 0.224, 0.225]

# Custom Image Dataset

In [ ]:
# Write a custom dataset class (inherits from torch.utils.data.Dataset)
class ImageFolderCustom(Dataset):

    # 1. Initialize with a targ_dir and transform (optional) parameter
    def __init__(self,
                 targ_dir: str,
                 path_df: DataFrame,
                 transform=None) -> None:

        # Get all image paths, classes
        self.img_df = path_df

        # Set all images to proper path
        self.img_df['path'] = self.check_path(targ_dir)

        self.paths = list(self.img_df['path'])

        # Setup transforms
        self.transform = transform

        self.classes, self.class_to_idx = self.find_classes()

    # 2. check if its already in proper format
    def check_path(self,
                   targ_dir: str) -> DataFrame:
        if str(targ_dir) in self.img_df.iloc[0,0]:
            return self.img_df['path'].astype('string')
        else:
            return str(targ_dir)+ '/' +  self.img_df['path'].astype('string')

    # 3. Make function to load images
    def load_image(self,
                   index: int) -> Image.Image:
        "Opens an image via a path and returns it."
        image_path = self.img_df.iloc[index, 0]
        return Image.open(image_path)

    # 4. Overwrite the __len__() method (optional but recommended for subclasses of torch.utils.data.Dataset)
    def __len__(self) -> int:
        "Returns the total number of samples."
        return self.img_df.shape[0]

    # 5. Overwrite the __getitem__() method (required for subclasses of torch.utils.data.Dataset)
    def __getitem__(self,
                    index: int) -> Tuple[torch.Tensor, int]:
        "Returns one sample of data, data and label (X, y)."
        img = self.load_image(index)
        class_name  = self.img_df.iloc[index, 1] # expects path in data_folder/class_name/image.jpeg
        class_idx = self.class_to_idx[class_name]

        # Transform if necessary
        if self.transform:
            return self.transform(img), class_idx # return data, label (X, y)
        else:
            return img, class_idx # return data, label (X, y)


    def find_classes(self) -> Tuple[List[str], Dict[str, int]]:

        col = self.img_df.columns
        # 1. Get the class names by scanning the target directory
        classes = sorted(self.img_df[col[1]].unique())

        # 2. Raise an error if class names not found
        if not classes:
            raise FileNotFoundError(f"Couldn't find any classes.")

        # 3. Create a dictionary of index labels (computers prefer numerical rather than string labels)
        class_to_idx = {cls_name: i for i, cls_name in enumerate(classes)}
        return classes, class_to_idx

In [ ]:
def create_dataloader(dir,
                      train_ds: DataFrame,
                      test_ds: DataFrame) -> Tuple[DataLoader[float], DataLoader[float], List[str]]:


    # train_transforms = transforms.Compose([
    #     #transforms.Resize((64, 64)),
    #     transforms.RandomHorizontalFlip(p=0.5),
    #     #transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    #     transforms.ToTensor()
    # ])


    manual_transforms = transforms.Compose([
        transforms.Resize((224, 224)), # 1. Reshape all images to 224x224 (though some models may require different sizes)
        transforms.ToTensor(), # 2. Turn image values to between 0 & 1
        transforms.Normalize(mean = MEAN, # 3. A mean of [0.485, 0.456, 0.406] (across each colour channel)
                         std = STD)
    ])



    # test_transforms = transforms.Compose([
    #     #transforms.Resize((64, 64)),
    #     transforms.ToTensor(),
    # ])


    train_data = ImageFolderCustom(targ_dir = dir,
                                          path_df = train_ds,
                                          transform= manual_transforms)

    test_data = ImageFolderCustom(targ_dir = dir,
                                          path_df = test_ds,
                                          transform= manual_transforms)



    classes, class_to_idx = train_data.find_classes()

    # Turn train, test and eval Dataset's into DataLoader's

    train_dataloader = DataLoader(dataset=train_data, # use custom created train Dataset
                                        batch_size=BATCH_SIZE, # how many samples per batch?
                                        num_workers=0, # how many subprocesses to use for data loading? (higher = more)
                                        shuffle=True) # shuffle the data?

    test_dataloader = DataLoader(dataset=test_data, # use custom created test Dataset
                                        batch_size=BATCH_SIZE,
                                        num_workers=0,
                                        shuffle=False) # don't usually need to shuffle testing data

    return train_dataloader, test_dataloader, classes

In [ ]:
train_dataloader, test_dataloader, classes = create_dataloader(dir=image_dir, train_ds=train_ds, test_ds=test_ds)

<ipython-input-9-b25ec1bd1d1a>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.img_df['path'] = self.check_path(targ_dir)


In [ ]:
train_dataloader, test_dataloader, classes

(<torch.utils.data.dataloader.DataLoader at 0x7ff1cdc2a770>,
 ['OSCC', 'With dysplasia', 'Without dysplasia'])

# Model Class

In [ ]:
class Engine():

    wandb.init(
    project="ressnet50-v1",
    config={
            "epochs": EPOCH,
            "batch_size": BATCH_SIZE,
            "lr": ALPHA,
            "architecture": "CNN",
            })




    def __init__(self,
                 model,
                 loss_fn,
                 optimizer,
                 device,
                 early_stopping = False):

        self.model = model
        self.loss_fn = loss_fn
        self.optimizer = optimizer
        self.device = device

        self.early_stopping = early_stopping
        self.counter = 0
        self.early_stop = False # type: ignore
        self.best_score = None

        # Copy your config
        self.config = wandb.config

    def model_summary(self,
                      input_size,
                      col_width = 20,):

        return summary(self.model,
                input_size = input_size, # make sure this is "input_size", not "input_shape" (batch_size, color_channels, height, width)
                col_names = ["input_size", "output_size", "num_params", "trainable"],
                col_width = col_width,
                row_settings = ["var_names"])

    def check_early_stop(self,
                   val_loss,
                   delta,
                   verbose,
                   patience):

        score = -val_loss
        # print(verbose)
        if self.best_score is None:
            self.best_score = score

        elif score < self.best_score + delta:
            self.counter += 1

            if verbose:
                print(f"Early stopping counter: {self.counter} out of {patience}")

            if self.counter >= patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.early_stop = False
            self.counter = 0


    def train_step(self, epoch):

        y_tr_true, y_tr_pred= [], []
        # Put model in train mode
        self.model.train()

        # Setup train loss and train accuracy values
        train_loss, train_acc = 0, 0

        n_steps_per_epoch = math.ceil(len(self.train_dataloader) / self.config.batch_size)

        example_ct = 0
        # Loop through data loader data batches
        for i, data in enumerate(tqdm(self.train_dataloader, desc=f'Epoch {epoch + 1}/{self.epochs}', unit='batch')):
            # Send data to target device
            X_train, y_train = data

            y_tr_true.extend(y_train) # collect all training labels

            X_train = X_train.to(self.device)
            y_train = y_train.to(self.device)

            # 1. Forward pass

            outputs = self.model(X_train)
            _, y_pred = torch.max(outputs, 1) # max along an axis gives both max val as well as index
            y_tr_pred.extend(y_pred.data.cpu())

            # 2. Calculate  and accumulate loss
            loss = self.loss_fn(outputs, y_train)

            train_loss += loss.item()

            # 3. Optimizer zero grad
            self.optimizer.zero_grad()

            # 4. Loss backward
            loss.backward()

            # 5. Optimizer step
            self.optimizer.step()


        # Adjust metrics to get average loss and accuracy per batch
        train_loss /= len(self.train_dataloader)
        train_acc = accuracy_score(y_tr_true, y_tr_pred)



        # Log Training info in W&B
        metrics ={
                      "train/train_loss": train_loss,
                      "train/epoch": (i + 1 + (n_steps_per_epoch * epoch)) / n_steps_per_epoch,
                      "train/train_acc": train_acc,
                  }

        wandb.log(metrics)

        return train_loss, train_acc


    def test_step(self, epoch):

        y_ts_true,  y_ts_pred = [], []

        # Put model in eval mode

        self.model.eval()

        # Setup test loss and test accuracy values
        test_loss, test_acc = 0, 0

        # Turn on inference context manager
        with torch.inference_mode():
            # Loop through DataLoader batches
            for i, data in enumerate(tqdm(self.test_dataloader, desc=f'Epoch {epoch + 1}/{self.epochs}', unit='batch')):
                # Send data to target device
                X_test, y_test = data
                y_ts_true.extend(y_test) # collect all training labels

                y_test = y_test.to(self.device)
                X_test = X_test.to(self.device)

                # 1. Forward pass
                outputs = self.model(X_test)
                _, y_pred = torch.max(outputs, 1) # max along an axis gives both max val as well as index

                y_ts_pred.extend(y_pred.data.cpu())

                # 2. Calculate and accumulate loss
                loss = self.loss_fn(outputs, y_test)
                test_loss += loss.item()

                # Calculate and accumulate accuracy


        # Adjust metrics to get average loss and accuracy per batch
        test_loss/= len(self.test_dataloader)
        test_acc = accuracy_score(y_ts_true, y_ts_pred)



        # Log Testing info in W&B
        metrics ={
                  "test/test_loss": test_loss,
                  "test/test_acc": test_acc,
                  }

        wandb.log(metrics)

        return test_loss, test_acc


    def train(self,
              train_dataloader,
              test_dataloader,
              epochs=1,
              delta = 0,
              patience = 10,
              verbose = False):

        self.epochs = epochs
        self.train_dataloader = train_dataloader
        self.test_dataloader = test_dataloader


        # Create empty results dictionary
        results = {"epoch":[],
                "train_loss": [],
                "train_acc": [],
                "test_loss": [],
                "test_acc": []
        }

        # Make sure model on target device
        self.model.to(self.device)

        # Loop through training and testing steps for a number of epochs
        for epoch in range(self.epochs):
            train_loss, train_acc = self.train_step(epoch)
            test_loss, test_acc = self.test_step(epoch)

            # Print out what's happening
            print(
            f"Epoch: {epoch+1} | "
            f"train_loss: {train_loss:.4f} | "
            f"train_acc: {train_acc:.4f} | "
            f"test_loss: {test_loss:.4f} | "
            f"test_acc: {test_acc:.4f}"
            )

            # Update results dictionary
            results["epoch"].append(epoch+1)
            results["train_loss"].append(train_loss)
            results["test_loss"].append(test_loss)
            results["train_acc"].append(train_acc)
            results["test_acc"].append(test_acc)



            if self.early_stopping:
                self.check_early_stop(test_loss, delta, verbose, patience)
                if self.early_stop:
                    print("Early Stopping")
                    break

        # Mark the run as finished
        wandb.finish()
        # Return the filled results at the end of the epochs
        return results


wandb: Currently logged in as: jhashulabh. Use `wandb login --relogin` to force relogin


# Train

In [ ]:
model = models.resnet50(pretrained=True).to(device)

# for param in model.parameters():
#     param.requires_grad = False

model.fc = nn.Sequential(
               nn.Linear(2048, 128),
               nn.ReLU(inplace=True),
               nn.Linear(128, 3)).to(device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:01<00:00, 99.5MB/s]


In [ ]:
loss_fn = nn.CrossEntropyLoss(reduction='sum') # computes the cross entropy loss between input logits and target.

optimizer = torch.optim.Adam(model.parameters(), lr = ALPHA)

# Start the timer
from timeit import default_timer as timer
start_time = timer()

# Setup training and save the results
Engine = Engine(model=model, loss_fn=loss_fn, optimizer=optimizer, device=device, early_stopping=True)
result = Engine.train(train_dataloader=train_dataloader, test_dataloader=test_dataloader, epochs=EPOCH, verbose=True)

# End the timer and print out how long it took
end_time = timer()
print(f"[INFO] Total training time: {end_time-start_time:.3f} seconds")

Epoch 1/50:   0%|          | 0/98 [00:00<?, ?batch/s]

In [ ]:
type(model), type(loss_fn), type(optimizer)

In [ ]:
result_df = pd.DataFrame(result)
result_df.head()

In [ ]:
result_df.shape

In [ ]:
# # parameters for Matplotlib
# params = {'legend.fontsize': 'x-large',
#           'figure.figsize': (15, 10),
#           'axes.labelsize': 'x-large',
#           'axes.titlesize':'x-large',
#           'xtick.labelsize':'x-large',
#           'ytick.labelsize':'x-large'
#          }

# CMAP = plt.cm.coolwarm

# plt.rcParams.update(params)

# Plot hist

In [ ]:
###-----------------------------------
### Function to plot Loss Curve
###-----------------------------------

def plot_torch_hist(hist_df : pd.DataFrame):
    '''
    Args:
      hist_df : pandas Dataframe with five columns
                First column need to be epoch, 'x' values
    '''
    # instantiate figure
    fig, axes = plt.subplots(1,2 , figsize = (15,6))

    facecolor = 'cyan'
    fontsize=12

    # Get columns by index to eliminate any column naming error
    x = "epoch"
    y1 = "train_loss"
    y2 = "test_loss"
    y3 = "train_acc"
    y4 = "test_acc"


    # properties  matplotlib.patch.Patch
    props = dict(boxstyle='round', facecolor=facecolor, alpha=0.5)

    # Where was min loss
    best = hist_df[hist_df[y2] == hist_df[y2].min()]

    # pick first axis
    ax = axes[0]

    # Plot all losses
    hist_df.plot(x = x, y = [y1,y2], ax = ax)

    # little beautification
    txtFmt = "Loss: \n  train: {:6.4f}\n   test: {:6.4f}"
    txtstr = txtFmt.format(hist_df.iloc[-1][y1],
                           hist_df.iloc[-1][y2]) #text to plot

    # place a text box in upper middle in axes coords
    ax.text(0.3, 0.95, txtstr,
            transform=ax.transAxes,
            fontsize=fontsize,
            verticalalignment='top',
            bbox=props)

    # Mark arrow at lowest
    offset = (best[y2].max() - best[y2].max())/10
    ax.annotate(f'Min: {best[y2].to_numpy()[0]:6.4f}', # text to print
                xy=(best[x].to_numpy(), best[y2].to_numpy()[0]), # Arrow start
                xytext=(best[x].to_numpy()-2, best[y2].to_numpy()[0]+offset), # location of text
                fontsize=fontsize, va='bottom', ha='right',bbox=props, # beautification of text
                arrowprops=dict(facecolor=facecolor, shrink=0.05)) # arrow

    # Draw vertical line at best value
    ax.axvline(x = best[x].to_numpy(),
               color = 'green',
               linestyle='-.', lw = 3);

    ax.set_xlabel(x.capitalize())
    ax.set_ylabel(y1.capitalize())
    ax.set_title('Errors')
    ax.grid()
    ax.legend(loc = 'upper left') # model legend to upper left

    # pick second axis
    ax = axes[1]

    # Plot accuracy
    hist_df.plot(x = x, y = [y3, y4], ax = ax)

    # little beautification
    txtFmt = "Accuracy: \n  train: {:6.4f}\n  test:  {:6.4f}"
    txtstr = txtFmt.format(hist_df.iloc[-1][y3],
                           hist_df.iloc[-1][y4]) #text to plot

    # place a text box in lower middle in axes coords
    ax.text(0.3, 0.2, txtstr,
            transform=ax.transAxes, fontsize=fontsize,
            verticalalignment='top', bbox=props)

    # Mark arrow at lowest
    offset = (best[y4].max() - best[y4].min())/10
    ax.annotate(f'Best: {best[y4].to_numpy()[0]:6.4f}', # text to print
                xy=(best[x].to_numpy(), best[y4].to_numpy()[0]), # Arrow start
                xytext=(best[x].to_numpy()-2, best[y4].to_numpy()[0]-offset), # location of text
                fontsize=fontsize, va='bottom', ha='right',bbox=props, # beautification of text
                arrowprops=dict(facecolor=facecolor, shrink=0.05)) # arrow


    # Draw a vertical line at best value
    ax.axvline(x = best[x].to_numpy(),
               color = 'green',
               linestyle='-.', lw = 3)

    # Labels
    ax.set_xlabel(x.capitalize())
    ax.set_ylabel(y3.capitalize())
    ax.set_title('Accuracies')
    ax.grid();
    ax.legend(loc = 'lower left')

    plt.tight_layout()




    # pick first axis
    ax = axes[0]

    # Plot all losses
    hist_df.plot(x = x, y = [y1,y2], ax = ax)

    # little beautification
    txtFmt = "Loss: \n  train: {:6.4f}\n   test: {:6.4f}"
    txtstr = txtFmt.format(hist_df.iloc[-1][y1],
                           hist_df.iloc[-1][y2]) #text to plot

    # place a text box in upper middle in axes coords
    ax.text(0.3, 0.95, txtstr,
            transform=ax.transAxes,
            fontsize=fontsize,
            verticalalignment='top',
            bbox=props)

    # Mark arrow at lowest
    offset = (best[y2].max() - best[y2].max())/10
    ax.annotate(f'Min: {best[y2].to_numpy()[0]:6.4f}', # text to print
                xy=(best[x].to_numpy(), best[y2].to_numpy()[0]), # Arrow start
                xytext=(best[x].to_numpy()-2, best[y2].to_numpy()[0]+offset), # location of text
                fontsize=fontsize, va='bottom', ha='right',bbox=props, # beautification of text
                arrowprops=dict(facecolor=facecolor, shrink=0.05)) # arrow

    # Draw vertical line at best value
    ax.axvline(x = best[x].to_numpy(),
               color = 'green',
               linestyle='-.', lw = 3);

    ax.set_xlabel(x.capitalize())
    ax.set_ylabel(y1.capitalize())
    ax.set_title('Errors')
    ax.grid()
    ax.legend(loc = 'upper left') # model legend to upper left

    # pick second axis
    ax = axes[1]

    # Plot accuracy
    hist_df.plot(x = x, y = [y3, y4], ax = ax)

    # little beautification
    txtFmt = "Accuracy: \n  train: {:6.4f}\n  test:  {:6.4f}"
    txtstr = txtFmt.format(hist_df.iloc[-1][y3],
                           hist_df.iloc[-1][y4]) #text to plot

    # place a text box in lower middle in axes coords
    ax.text(0.3, 0.2, txtstr,
            transform=ax.transAxes, fontsize=fontsize,
            verticalalignment='top', bbox=props)

    # Mark arrow at lowest
    offset = (best[y4].max() - best[y4].min())/10
    ax.annotate(f'Best: {best[y4].to_numpy()[0]:6.4f}', # text to print
                xy=(best[x].to_numpy(), best[y4].to_numpy()[0]), # Arrow start
                xytext=(best[x].to_numpy()-2, best[y4].to_numpy()[0]-offset), # location of text
                fontsize=fontsize, va='bottom', ha='right',bbox=props, # beautification of text
                arrowprops=dict(facecolor=facecolor, shrink=0.05)) # arrow


    # Draw a vertical line at best value
    ax.axvline(x = best[x].to_numpy(),
               color = 'green',
               linestyle='-.', lw = 3)

    # Labels
    ax.set_xlabel(x.capitalize())
    ax.set_ylabel(y3.capitalize())
    ax.set_title('Accuracies')
    ax.grid();
    ax.legend(loc = 'lower left')

    plt.tight_layout()

In [ ]:
plot_torch_hist(result_df)